In [1]:
import os
import transformers
import haystack
import torch
import datasets
import importlib

/home/power/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(haystack.EMBED_MODEL)

json_config = {
  "attention_bias": False,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 8,
  "num_hidden_layers": 18,
  "num_key_value_heads": 1,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "rope_scaling": None,
  "rope_theta": 10000.0,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.0.dev0",
  "use_cache": True,
  "vocab_size": tokenizer.vocab_size
}

config = haystack.HaystackConfig.from_dict(json_config)
model = haystack.HaystackForCausalLM(config)
model.num_parameters() / 1e9

0.182852992

In [5]:
block_size = 200
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


def load_dataset(split):
  dataset = datasets.load_dataset(haystack.DATASET, split=split, cache_dir=haystack.CACHE_DIR, streaming=True)
  tokenized_dataset = dataset.map(
      lambda examples: tokenizer(examples["text"], padding="max_length", truncation=True, max_length=block_size),
      batched=True,
      remove_columns=dataset.features.keys(),
  )
  return tokenized_dataset.map(
      group_texts,
      batched=True,
      batch_size=32,
  )

lm_train, lm_test = load_dataset("train"), load_dataset("validation")
lm_test = lm_test.take(100)

/home/power/miniconda3/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for allenai/peS2o contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/peS2o
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [12]:
from transformers import Trainer, TrainingArguments

lm_train = lm_train.take(1000)

model_name = 'haystack-v0'
training_args = TrainingArguments(
    f"{model_name}-pretrain-scholar",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    optim="adamw_torch",
    per_device_train_batch_size=1,
    fp16=True,
    max_steps=10000,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_train,
    eval_dataset=lm_test,
)
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 